In [10]:
!pip install cryptography


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\USER PC\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import pymysql
import pandas as pd
from datetime import datetime

In [2]:
#first sales_1
df1 = pd.read_csv('store_sales_1.csv')
df2 = pd.read_csv('store_sales_2.csv')
df3 = pd.read_csv('store_sales_3.csv')

df = pd.concat([df1, df2, df3], ignore_index=True)
df

,ProductName,Qty,Unit_Price,SaleDate,CurrencyType,CustomerID,StoreID
0,Smith Paper,3.0,10.50,7/13/2024,OMR,9ca482a2-0356-49c1-b5e3-88ae98d1cc2f,Store_A
1,Johnson Screen,NaN,NaN,2/23/2025,Usd,c0b9df4e-8f03-4bf0-a31b-0a7d7c2a8907,Store_A
2,Roberts Ingredient,3.0,30.00,11/13/2024,USD,97dc18e3-2c12-4e26-9863-32514e82e822,Store_A
3,White Monitor,NaN,10.50,4/16/2025,USD,e4d09733-d496-47b3-a4b5-04de84d8fd06,Store_A
4,Rodriguez Keyboard,2.0,20.00,8/3/2024,usd,435ecb46-4545-4af7-b72c-119f64d193a5,Store_A
...,...,...,...,...,...,...,...
295,Case,1.0,30.00,8/22/2024,Usd,b0552606-1fe5-420f-abf0-a86289c250ba,STORE_A
296,Black,NaN,30.00,12/24/2024,Usd,6da0ca26-e56e-41e1-89b0-004aa6d74b3e,STORE_A
297,On,1.0,20.00,9/6/2024,NaN,045081fa-0df3-4904-820b-067b4176fb85,STORE_A
298,Soon,2.0,NaN,7/4/2024,NaN,a6b5132d-2026-4400-83d9-a64ed546bd51,STORE_A


In [3]:
df.isnull().sum()

ProductName      0
Qty             73
Unit_Price      67
SaleDate         0
CurrencyType    54
CustomerID      31
StoreID          0
dtype: int64

In [16]:
# 1. Drop rows with missing CustomerID

df = df.dropna(subset=['CustomerID']).copy()


# 2. Fill missing values

# Numeric columns
df['Qty'] = df['Qty'].fillna(df['Qty'].median())
df['Unit_Price'] = df['Unit_Price'].fillna(df['Unit_Price'].median())

# Categorical column
df['CurrencyType'] = df['CurrencyType'].fillna('OMR')  # default to OMR if missing


# 3. Convert types

# Qty → int
df['Qty'] = df['Qty'].astype(int)

# SaleDate → datetime
df['SaleDate'] = pd.to_datetime(df['SaleDate'], errors='coerce', dayfirst=True)


# 4. Create Total_Price column

df['Total_Price'] = df['Qty'] * df['Unit_Price']


# 5. Convert prices to OMR safely

# Standardize CurrencyType

df['CurrencyType'] = df['CurrencyType'].str.upper()  # USD, EUR, OMR, etc.


# Define exchange rates to OMR

exchange_rates = {
    'USD': 0.385,
    'EUR': 0.41,
    'OMR': 1
}

# Map rates, unknown currencies assume 1 (already OMR)
df['rate_to_OMR'] = df['CurrencyType'].map(exchange_rates).fillna(1)


# Convert Unit_Price and Total_Price

df['Unit_Price'] = df['Unit_Price'] * df['rate_to_OMR']
df['Total_Price'] = df['Total_Price'] * df['rate_to_OMR']


# Drop helper column

df = df.drop(columns=['rate_to_OMR'])


# Round prices to 3 decimals (standard for OMR)

df['Unit_Price'] = df['Unit_Price'].round(3)
df['Total_Price'] = df['Total_Price'].round(3)

df[['CurrencyType', 'Unit_Price', 'Total_Price']].head()

#store_id
import re

def normalize_store(value):
    value = str(value).strip()

    match = re.search(r'(store)[\s\-_]*([a-zA-Z0-9]+)', value, re.IGNORECASE)
    if match:
        letter = match.group(2)[0].upper() 
        return f"Store_{letter}"

    return "Store_X"  

df['StoreID'] = df['StoreID'].apply(normalize_store)


print(df.isnull().sum())
print(df.dtypes)
df.head()

ProductName     0
Qty             0
Unit_Price      0
SaleDate        0
CurrencyType    0
CustomerID      0
StoreID         0
Total_Price     0
dtype: int64
ProductName             object
Qty                      int64
Unit_Price             float64
SaleDate        datetime64[ns]
CurrencyType            object
CustomerID              object
StoreID                 object
Total_Price            float64
dtype: object


,ProductName,Qty,Unit_Price,SaleDate,CurrencyType,CustomerID,StoreID,Total_Price
0,Smith Paper,3,10.500,2024-07-13,OMR,9ca482a2-0356-49c1-b5e3-88ae98d1cc2f,Store_A,31.500
1,Johnson Screen,2,0.346,2025-02-23,USD,c0b9df4e-8f03-4bf0-a31b-0a7d7c2a8907,Store_A,0.692
2,Roberts Ingredient,3,0.659,2024-11-13,USD,97dc18e3-2c12-4e26-9863-32514e82e822,Store_A,1.977
3,White Monitor,2,0.231,2025-04-16,USD,e4d09733-d496-47b3-a4b5-04de84d8fd06,Store_A,0.461
4,Rodriguez Keyboard,2,0.439,2024-08-03,USD,435ecb46-4545-4af7-b72c-119f64d193a5,Store_A,0.879


In [17]:
# STEP 1 — CONNECT
sales_connection = pymysql.connect(
    host="localhost",
    user="root",
    password="Oman99690050#",
    charset="utf8mb4",
    autocommit=True
)
cursor = sales_connection.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS Sales1;")
cursor.execute("USE Sales1;")



# STEP 2 — CREATE TABLES
cursor.execute("""
CREATE TABLE IF NOT EXISTS Products (
    product_id INT AUTO_INCREMENT PRIMARY KEY,
    ProductName VARCHAR(255),
    Unit_Price FLOAT,
    CurrencyType VARCHAR(10)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Customers (
    customer_id VARCHAR(50) PRIMARY KEY
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Stores (
    store_id VARCHAR(50) PRIMARY KEY
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Sales (
    sale_id INT AUTO_INCREMENT PRIMARY KEY,
    SaleDate DATETIME,
    Qty INT,
    Total_Price FLOAT,
    product_id INT,
    customer_id VARCHAR(50),
    store_id VARCHAR(50),
    FOREIGN KEY (product_id) REFERENCES Products(product_id),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id),
    FOREIGN KEY (store_id) REFERENCES Stores(store_id)
);
""")

# STEP 3 — PREPARE DATA
products_df = df[['ProductName', 'Unit_Price', 'CurrencyType']].drop_duplicates()
customers_df = df[['CustomerID']].drop_duplicates()
stores_df = df[['StoreID']].drop_duplicates()

# STEP 4 — INSERT PRODUCTS (ROW-BY-ROW)
product_id_map = {}

for _, row in products_df.iterrows():
    cursor.execute("""
        INSERT INTO Products (ProductName, Unit_Price, CurrencyType)
        VALUES (%s, %s, %s)
    """, (row['ProductName'], row['Unit_Price'], row['CurrencyType']))

    product_id_map[(row['ProductName'], row['Unit_Price'], row['CurrencyType'])] = cursor.lastrowid

# STEP 5 — INSERT CUSTOMERS (BULK)
cursor.executemany("""
    INSERT IGNORE INTO Customers (customer_id)
    VALUES (%s)
""", [(x,) for x in customers_df['CustomerID']])

# STEP 6 — INSERT STORES (BULK)
cursor.executemany("""
    INSERT IGNORE INTO Stores (store_id)
    VALUES (%s)
""", [(x,) for x in stores_df['StoreID']])

cursor.close()
sales_connection.close()

print("ok")

ok


In [18]:
# STEP 7 — CONNECT AGAIN
sales_connection = pymysql.connect(
    host="localhost",
    user="root",
    password="Oman99690050#",
    database="Sales1",
    autocommit=True
)
cursor = sales_connection.cursor()

# Rebuild product_id_map from database (important)
cursor.execute("SELECT product_id, ProductName, Unit_Price, CurrencyType FROM Products")
rows = cursor.fetchall()

product_id_map = {}
for r in rows:
    product_id_map[(r[1], r[2], r[3])] = r[0]

# STEP 8 — INSERT SALES
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO Sales (SaleDate, Qty, Total_Price, product_id, customer_id, store_id)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        row['SaleDate'],
        row['Qty'],
        row['Total_Price'],
        product_id_map[(row['ProductName'], row['Unit_Price'], row['CurrencyType'])],
        row['CustomerID'],
        row['StoreID']
    ))

cursor.close()
sales_connection.close()

print("STEP 7-8 completed — Sales inserted.")


STEP 7-8 completed — Sales inserted.


In [15]:
df

,ProductName,Qty,Unit_Price,SaleDate,CurrencyType,CustomerID,StoreID,Total_Price
0,Smith Paper,3,10.500,2024-07-13,OMR,9ca482a2-0356-49c1-b5e3-88ae98d1cc2f,STORE_A,31.500
1,Johnson Screen,2,0.899,2025-02-23,USD,c0b9df4e-8f03-4bf0-a31b-0a7d7c2a8907,STORE_A,1.798
2,Roberts Ingredient,3,1.712,2024-11-13,USD,97dc18e3-2c12-4e26-9863-32514e82e822,STORE_A,5.136
3,White Monitor,2,0.599,2025-04-16,USD,e4d09733-d496-47b3-a4b5-04de84d8fd06,STORE_A,1.199
4,Rodriguez Keyboard,2,1.141,2024-08-03,USD,435ecb46-4545-4af7-b72c-119f64d193a5,STORE_A,2.282
...,...,...,...,...,...,...,...,...
295,Case,1,1.712,2024-08-22,USD,b0552606-1fe5-420f-abf0-a86289c250ba,STORE_A,1.712
296,Black,2,1.712,2024-12-24,USD,6da0ca26-e56e-41e1-89b0-004aa6d74b3e,STORE_A,3.424
297,On,1,20.000,2024-09-06,OMR,045081fa-0df3-4904-820b-067b4176fb85,STORE_A,20.000
298,Soon,2,15.750,2024-07-04,OMR,a6b5132d-2026-4400-83d9-a64ed546bd51,STORE_A,31.500


In [19]:
# Question 1: Top 5 Products by Total Revenue
# -------------------------------

top_products = (
    df.groupby("ProductName")["Total_Price"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
)

print("Top 5 Products by Total Revenue:")
print(top_products)
print("\n")

Top 5 Products by Total Revenue:
ProductName
Moore Chair       131.038
Gray Tool         111.346
Mitchell Frame    110.250
Griffin Cream      92.076
Flores Coat        90.879
Name: Total_Price, dtype: float64




In [20]:
# Question 2: Top 5 Customers by Spending
# -------------------------------

top_customers = (
    df.groupby("CustomerID")["Total_Price"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
)

print("Top 5 Customers by Spending:")
print(top_customers)
print("\n")

Top 5 Customers by Spending:
CustomerID
0822d2d7-9454-4aa3-95bc-b869d6957852    90.0
07991be5-0500-4bdc-95bc-a3ee50f4d268    90.0
d9966648-3b63-487c-9011-d52513c114d7    90.0
0b4a7550-cb81-475b-b8f0-4d4e9668f7e8    90.0
ce05d98c-c6f0-4f3c-b889-ec3ccb2da92f    90.0
Name: Total_Price, dtype: float64




In [21]:
# Question 3: Top 5 Stores by Sales Revenue
# -------------------------------

top_stores = (
    df.groupby("StoreID")["Total_Price"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
)

print("Top 5 Stores by Sales Revenue:")
print(top_stores)

Top 5 Stores by Sales Revenue:
StoreID
Store_A    3599.94
Name: Total_Price, dtype: float64
